In [8]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
import h5py

In [2]:
def filter_boxes(confidence, bbox_coordinates, class_probs, threshold=0.6):
    """
    Filters bounding boxes based on class confidence and a threshold.
    
    Arguments:
    confidence -- tensor of shape (19, 19, 5, 1), representing the probability that an object appears in each box.
    bbox_coordinates -- tensor of shape (19, 19, 5, 4), containing each box's coordinates (x_center, y_center, width, height).
    class_probs -- tensor of shape (19, 19, 5, 80), containing the class probabilities for each box.
    threshold -- float, threshold for class scores (default is 0.6). Only boxes with a class score >= threshold are kept.
    
    Returns:
    class_ -- tensor of shape (None,), containing the index of the class with the highest score for each selected box.
    bbox_coords -- tensor of shape (None, 4), containing the coordinates (x_center, y_center, width, height) of the selected boxes.
    class_score -- tensor of shape (None,), containing the class score for the selected boxes.
    """
    
    # scores of 80 classes of 5 boxes of 19x19 grid cells
    class_scores = confidence * class_probs  # shape (19, 19, 5, 80)
    
    # index of the class having the max score for each box 
    class_ = tf.math.argmax(class_scores, axis=-1)  # shape (19, 19, 5)
    # the score for that class
    class_score = tf.math.reduce_max(class_scores, axis=-1)  # shape (19, 19, 5)

    # True if the score of a box >= threshold
    filtering_mask = class_score >= threshold  # shape (19, 19, 5)

    # Pick boxes that have their class's scores >= threshold
    class_ = tf.boolean_mask(class_, filtering_mask) 
    class_score = tf.boolean_mask(class_score, filtering_mask)
    bbox_coords = tf.boolean_mask(bbox_coordinates, filtering_mask)

    return class_, bbox_coords, class_score

In [3]:
# Example
class_ = tf.random.uniform(shape=(2, 2, 5,), minval=0, maxval=80, dtype=tf.int32)
class_score = tf.random.uniform(shape=(2, 2, 5,), minval=0, maxval=1)
bbox_coords = tf.random.uniform(shape=(2, 2, 5, 4), minval=0, maxval=19)
threshold = 0.8

# Round tensors to 2 decimal places
class_score = tf.round(class_score * 100) / 100
bbox_coords = tf.round(bbox_coords * 100) / 100

# Create a filtering mask based on the threshold
filtering_mask = class_score >= threshold

# Apply the mask
filtered_class_ = tf.boolean_mask(class_, filtering_mask)
filtered_class_score = tf.boolean_mask(class_score, filtering_mask)
filtered_bbox_coords = tf.boolean_mask(bbox_coords, filtering_mask)

# Print results
print("CLASSES:\n", class_)
print("CLASS_SCORES:\n", class_score)
print("BBOX_COORDINATES:\n", bbox_coords)
print()
print("MASK:\n", filtering_mask)
print()
print("FILTERED CLASSES:\n", filtered_class_)
print("FILTERED CLASS_SCORES:\n", filtered_class_score)
print("FILTERED BBOX_COORDINATES:\n", filtered_bbox_coords)

CLASSES:
 tf.Tensor(
[[[55 73 20 57  1]
  [14 19 13 20 33]]

 [[22 21 12 27 16]
  [22 32 74 64 34]]], shape=(2, 2, 5), dtype=int32)
CLASS_SCORES:
 tf.Tensor(
[[[0.91 0.73 0.13 0.95 0.47]
  [0.09 0.77 0.05 0.53 0.42]]

 [[0.03 0.84 0.21 0.71 0.25]
  [0.29 0.31 0.31 0.42 0.63]]], shape=(2, 2, 5), dtype=float32)
BBOX_COORDINATES:
 tf.Tensor(
[[[[11.75 15.11  5.87  6.  ]
   [ 5.69 17.48 10.87  8.05]
   [ 6.31 13.12  7.43 13.34]
   [10.2  10.95 12.48  8.3 ]
   [18.48 12.42 17.74 13.17]]

  [[16.3   3.77  4.47 12.15]
   [11.91  2.1  17.17  2.06]
   [ 5.52  0.05 10.27  5.57]
   [12.6   3.45  8.1   0.22]
   [ 8.65  0.13 16.86  3.44]]]


 [[[ 4.31  9.28 10.64 17.5 ]
   [11.42  1.86  1.3   7.57]
   [ 3.36  3.49 12.94 16.04]
   [ 0.22  6.94  2.4   6.59]
   [ 2.67  7.28  4.57 12.38]]

  [[ 3.37  4.49  9.32  7.74]
   [ 2.94  2.7  15.78 10.79]
   [ 7.35 10.   12.8   6.86]
   [12.86 15.31  4.48  3.51]
   [ 5.53  4.36 10.93 15.92]]]], shape=(2, 2, 5, 4), dtype=float32)

MASK:
 tf.Tensor(
[[[ True Fals

# Intersection over Union

In [4]:
def compute_IoU(box_1, box_2):
    x1_1, y1_1, x2_1, y2_1 = box_1
    x1_2, y1_2, x2_2, y2_2 = box_2

    area_1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area_2 = (x2_2 - x1_2) * (y2_2 - y1_2)

    x1_inner = max(x1_1, x1_2)
    y1_inner = max(y1_1, y1_2)
    x2_inner = min(x2_1, x2_2)
    y2_inner = min(y2_1, y2_2)

    inner_width = max(0, x2_inner - x1_inner)  # to check if there is an intersection between 2 boxes
    inner_height = max(0, y2_inner - y1_inner)
    
    inner_area = inner_width * inner_height
    outer_area = area_1 + area_2 - inner_area

    iou = inner_area / outer_area
    return iou

In [ ]:
def suppress_nonMax(boxes, scores, classes, max_boxes=10, iou_threshold=0.5):
    

In [17]:
import tensorflow as tf

# Example bounding boxes: [y_min, x_min, y_max, x_max]
boxes = tf.constant([[0.1, 0.1, 0.5, 0.5],    # Box 1
                     [0.2, 0.2, 0.6, 0.6],    # Box 2 (overlaps with Box 1)
                     [0.7, 0.7, 1.0, 1.0]],   # Box 3 (no overlap)
                    dtype=tf.float32)

# Confidence scores for each box
scores = tf.constant([0.9, 0.75, 0.8], dtype=tf.float32)

# Max number of boxes to keep after NMS
max_boxes = 2

# Intersection over Union (IoU) threshold
iou_threshold = 0.5

# Apply Non-Max Suppression
nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes, iou_threshold)

# Gather the boxes that passed NMS
selected_boxes = tf.gather(boxes, nms_indices)
selected_scores = tf.gather(scores, nms_indices)

# Print the selected boxes and their scores
print("Selected Boxes:\n", selected_boxes)
print("Selected Scores:\n", selected_scores)


Selected Boxes:
 tf.Tensor(
[[0.1 0.1 0.5 0.5]
 [0.7 0.7 1.  1. ]], shape=(2, 4), dtype=float32)
Selected Scores:
 tf.Tensor([0.9 0.8], shape=(2,), dtype=float32)


In [18]:
tf.Variable(max_boxes)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=2>